In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/scareware/'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-03 16:12:20.982571: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 97 images belonging to 11 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(97, 11)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(67, 64, 64, 3)

In [6]:
X_test.shape

(30, 64, 64, 3)

In [7]:
y_train.shape

(67, 11)

In [8]:
y_test.shape

(30, 11)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 11 # Changed from 10 to 11 for families on scareware

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(16, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-03 16:12:24.037912: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-03 16:12:24.364372: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-03 16:12:24.364470: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-03 16:12:24.365975: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 16:12:24.384664: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-03 16:12:24.3860

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        4336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(67, 11)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([ 2,  9,  1,  7,  9,  7,  6,  4,  1,  0,  9,  9, 10,  3,  9,  3, 10,
        7,  7,  2,  0,  3,  5,  7,  7,  8,  6,  1,  7,  6,  8,  8,  8,  2,
        8,  9,  5,  4,  9,  6,  6,  5,  2,  3,  1,  7, 10,  1,  4,  1,  2,
       10,  4,  5,  1,  4,  1,  7,  6,  6, 10,  6,  2,  4,  9,  9,  3])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

2023-05-03 16:12:24.557378: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 1s 253ms/step - loss: 0.6230 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5069 - val_loss: 0.3911 - val_f1_m: 0.0588 - val_precision_m: 0.2500 - val_recall_m: 0.0333 - val_auc: 0.4875
Epoch 2/100
3/3 [==============================] - 0s 37ms/step - loss: 0.4158 - f1_m: 0.0139 - precision_m: 0.0292 - recall_m: 0.0091 - auc: 0.5240 - val_loss: 0.3280 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5355
Epoch 3/100
3/3 [==============================] - 0s 37ms/step - loss: 0.3787 - f1_m: 0.0443 - precision_m: 0.1486 - recall_m: 0.0260 - auc: 0.5122 - val_loss: 0.3165 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.6222
Epoch 4/100
3/3 [==============================] - 0s 39ms/step - loss: 0.3751 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4737 - val_loss: 0.3217 - val_f1_m: 0.0000e+0

3/3 [==============================] - 0s 38ms/step - loss: 0.2319 - f1_m: 0.2297 - precision_m: 0.6976 - recall_m: 0.1380 - auc: 0.8584 - val_loss: 0.2306 - val_f1_m: 0.5000 - val_precision_m: 1.0000 - val_recall_m: 0.3333 - val_auc: 0.8449
Epoch 32/100
3/3 [==============================] - 0s 38ms/step - loss: 0.2228 - f1_m: 0.2056 - precision_m: 0.6847 - recall_m: 0.1211 - auc: 0.9063 - val_loss: 0.2334 - val_f1_m: 0.4211 - val_precision_m: 1.0000 - val_recall_m: 0.2667 - val_auc: 0.8354
Epoch 33/100
3/3 [==============================] - 0s 43ms/step - loss: 0.2230 - f1_m: 0.3290 - precision_m: 0.9398 - recall_m: 0.2092 - auc: 0.8871 - val_loss: 0.2389 - val_f1_m: 0.2353 - val_precision_m: 1.0000 - val_recall_m: 0.1333 - val_auc: 0.8258
Epoch 34/100
3/3 [==============================] - 0s 41ms/step - loss: 0.2233 - f1_m: 0.3131 - precision_m: 0.8643 - recall_m: 0.1936 - auc: 0.9095 - val_loss: 0.2388 - val_f1_m: 0.2353 - val_precision_m: 1.0000 - val_recall_m: 0.1333 - val_auc: 

3/3 [==============================] - 0s 36ms/step - loss: 0.1050 - f1_m: 0.7692 - precision_m: 0.8395 - recall_m: 0.7114 - auc: 0.9784 - val_loss: 0.1442 - val_f1_m: 0.6923 - val_precision_m: 0.8182 - val_recall_m: 0.6000 - val_auc: 0.9497
Epoch 64/100
3/3 [==============================] - 0s 37ms/step - loss: 0.0831 - f1_m: 0.8828 - precision_m: 0.9491 - recall_m: 0.8268 - auc: 0.9946 - val_loss: 0.1466 - val_f1_m: 0.6792 - val_precision_m: 0.7826 - val_recall_m: 0.6000 - val_auc: 0.9421
Epoch 65/100
3/3 [==============================] - 0s 37ms/step - loss: 0.0929 - f1_m: 0.8276 - precision_m: 0.9315 - recall_m: 0.7487 - auc: 0.9882 - val_loss: 0.1566 - val_f1_m: 0.6800 - val_precision_m: 0.8500 - val_recall_m: 0.5667 - val_auc: 0.9346
Epoch 66/100
3/3 [==============================] - 0s 35ms/step - loss: 0.0678 - f1_m: 0.8935 - precision_m: 0.9197 - recall_m: 0.8698 - auc: 0.9949 - val_loss: 0.1716 - val_f1_m: 0.6667 - val_precision_m: 0.8095 - val_recall_m: 0.5667 - val_auc: 

3/3 [==============================] - 0s 37ms/step - loss: 0.0385 - f1_m: 0.9615 - precision_m: 0.9661 - recall_m: 0.9570 - auc: 0.9993 - val_loss: 0.1927 - val_f1_m: 0.6786 - val_precision_m: 0.7308 - val_recall_m: 0.6333 - val_auc: 0.9017
Epoch 96/100
3/3 [==============================] - 0s 37ms/step - loss: 0.0273 - f1_m: 0.9914 - precision_m: 1.0000 - recall_m: 0.9831 - auc: 0.9999 - val_loss: 0.1946 - val_f1_m: 0.6909 - val_precision_m: 0.7600 - val_recall_m: 0.6333 - val_auc: 0.9186
Epoch 97/100
3/3 [==============================] - 0s 36ms/step - loss: 0.0291 - f1_m: 0.9784 - precision_m: 0.9831 - recall_m: 0.9740 - auc: 0.9997 - val_loss: 0.1953 - val_f1_m: 0.6909 - val_precision_m: 0.7600 - val_recall_m: 0.6333 - val_auc: 0.9216
Epoch 98/100
3/3 [==============================] - 0s 37ms/step - loss: 0.0283 - f1_m: 0.9909 - precision_m: 0.9909 - recall_m: 0.9909 - auc: 0.9996 - val_loss: 0.1877 - val_f1_m: 0.6667 - val_precision_m: 0.7037 - val_recall_m: 0.6333 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 20ms/step - loss: 0.1707 - f1_m: 0.6786 - precision_m: 0.7308 - recall_m: 0.6333 - auc: 0.9146


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 20ms/step - loss: 0.1707 - f1_m: 0.6786 - precision_m: 0.7308 - recall_m: 0.6333 - auc: 0.9146


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.6785714030265808
